In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pickle

In [2]:
artists = pd.read_csv('artists.csv')

In [3]:
tracks = pd.read_csv('tracks.csv')

In [4]:
artists.head()

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0


In [5]:
tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [6]:
artists['genres_upd'] = artists['genres'].apply(lambda x:[re.sub(' ','_',i) for i in eval(x)])

In [7]:
artists = artists[artists['genres'] != "[]"]

In [8]:
artists['name_upd'] = artists['name'].apply(lambda x:re.sub(' ','_',x))

In [9]:
tracks['artists_upd'] = tracks['artists'].apply(lambda x:[re.sub(' ','_',i) for i in eval(x)])

In [10]:
tracks.dropna(inplace =True)

In [11]:
#need to create my own song identifier because there are duplicates of the same song with different ids. I see different
tracks['artists_song'] = tracks.apply(lambda row: row['artists_upd'][0]+row['name'],axis = 1)

In [12]:
tracks.sort_values(['popularity'], ascending = False, inplace = True)

In [13]:
tracks.drop_duplicates('artists_song',keep='first',inplace = True) #May drop songs with high popularity in the duplicates. Needs to be revisited

In [14]:
tracks

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd,artists_song
93802,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),100,198082,1,"['Justin Bieber', 'Daniel Caesar', 'Giveon']","['1uNFoZAHBGtllmzznpCI3s', '20wkVLutqVOYrc0kxF...",2021-03-19,0.677,0.6960,...,1,0.1190,0.3210,0.000000,0.420,0.464,90.030,4,"[Justin_Bieber, Daniel_Caesar, Giveon]",Justin_BieberPeaches (feat. Daniel Caesar & Gi...
93803,7lPN2DXiMsVn7XUKtOW1CS,drivers license,99,242014,1,['Olivia Rodrigo'],['1McMsnEElThX1knmY4oliG'],2021-01-08,0.585,0.4360,...,1,0.0601,0.7210,0.000013,0.105,0.132,143.874,4,[Olivia_Rodrigo],Olivia_Rodrigodrivers license
93804,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,98,132780,0,['Masked Wolf'],['1uU7g3DNSbsu0QjSEqZtEd'],2021-01-06,0.778,0.6950,...,0,0.0913,0.1750,0.000000,0.150,0.472,149.996,4,[Masked_Wolf],Masked_WolfAstronaut In The Ocean
92811,6tDDoYIxWvMLTdKpjFkc1B,telepatía,97,160191,0,['Kali Uchis'],['1U1el3k54VvEUzo3ybLPlM'],2020-12-04,0.653,0.5240,...,0,0.0502,0.1120,0.000000,0.203,0.553,83.970,4,[Kali_Uchis],Kali_Uchistelepatía
92810,5QO79kh1waicV47BqGRL3g,Save Your Tears,97,215627,1,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2020-03-20,0.680,0.8260,...,1,0.0309,0.0212,0.000012,0.543,0.644,118.051,4,[The_Weeknd],The_WeekndSave Your Tears
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511203,1X2NNzkNTgfZOV2HB4hycq,Kshamiyimpuma,0,192920,0,['Baby Saroja'],['5pu1G665WuW7lUsqJQDpLE'],1937-12-31,0.449,0.1390,...,1,0.0408,0.9480,0.001350,0.148,0.489,96.243,4,[Baby_Saroja],Baby_SarojaKshamiyimpuma
511201,1Sus6U34jUYRG0N0ZKCXRt,Birhan Ke Shringaar,0,195193,0,['Bina Chowdhury'],['4Wz0BlGS2iVtiKaAQ6Jku6'],1937-12-31,0.517,0.3330,...,0,0.0449,0.9920,0.002990,0.392,0.516,133.200,4,[Bina_Chowdhury],Bina_ChowdhuryBirhan Ke Shringaar
315090,3zC56J4xI5omH4oASfHIJt,"Brennið þið, vitar",0,259088,0,"['Páll Ísólfsson', 'Karlakórinn Fóstbræður', '...","['27DN50VakKxJ8rXGYAUNgj', '19OGX5U8snp3kgI2xK...",1969-01-01,0.169,0.1570,...,1,0.0348,0.9690,0.266000,0.672,0.150,98.543,4,"[Páll_Ísólfsson, Karlakórinn_Fóstbræður, Kór_Í...","Páll_ÍsólfssonBrennið þið, vitar"
315087,2aH4ntf7RtwaLt4MUQ3PPg,Morgunn í maí,0,140249,0,['Guðmundur Jónsson'],['2maaML9cG6rRknp161or5V'],1969-01-01,0.477,0.6370,...,0,0.0482,0.4050,0.000000,0.149,0.854,132.390,4,[Guðmundur_Jónsson],Guðmundur_JónssonMorgunn í maí


In [15]:
artists_exploded = tracks[['artists_upd','id']].explode('artists_upd')

In [16]:
artists_exploded_enriched = artists_exploded.merge(artists, how = 'left', left_on = 'artists_upd',right_on = 'name_upd')


In [17]:
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres_upd.isnull()]

In [18]:
artists_genres_consolidated = artists_exploded_enriched_nonnull.groupby('id_x')['genres_upd'].apply(list).reset_index()

In [19]:
artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['genres_upd'].apply(lambda x:list(set(list(np.concatenate(x)))))

In [20]:
tracks = artists_genres_consolidated.merge(tracks, how = 'left', left_on = 'id_x',right_on = 'id')


In [21]:
tracks.drop(['id'],axis =1 ,inplace = True)

In [22]:
tracks.rename(columns={'id_x': 'id'}, inplace=True)

In [23]:
tracks['year'] = tracks['release_date'].apply(lambda x: x.split('-')[0])

In [24]:
float_cols = list(tracks.dtypes[tracks.dtypes == 'float64'].index.values)

In [25]:
# create 3 point buckets for popularity 
tracks['popularity_red'] = tracks['popularity'].apply(lambda x: int(x/3))

In [26]:
tracks

,id,genres_upd,consolidates_genre_lists,name,popularity,duration_ms,explicit,artists,id_artists,release_date,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd,artists_song,year,popularity_red
0,0004Uy71ku11n3LMpuyf59,[[polish_rock]],[polish_rock],24.11.94 - Wersja Akustyczna,34,258907,0,['Golden Life'],['3NyQ2NjqMjDtVSiEkXFW2r'],2012-01-01,...,0.177000,0.001480,0.0748,0.381,140.056,4,[Golden_Life],Golden_Life24.11.94 - Wersja Akustyczna,2012,11
1,000CSYu4rvd8cQ7JilfxhZ,"[[country_quebecois, rock_quebecois]]","[rock_quebecois, country_quebecois]",Adam et Ève,43,189107,0,['Kaïn'],['1KMHmX2PgvEl1nFzFmEdcZ'],2005-05-03,...,0.345000,0.000000,0.1130,0.548,95.128,4,[Kaïn],KaïnAdam et Ève,2005,14
2,000DsoWJKHdaUmhgcnpr8j,[[barnmusik]],[barnmusik],Balla Trazan Apansson / Vegetarianvisan,16,234533,0,['Trazan & Banarne'],['5PbCQPKjb0wQwaE7i5OVC6'],1977-07-11,...,0.642000,0.000220,0.3440,0.559,106.453,4,[Trazan_&_Banarne],Trazan_&_BanarneBalla Trazan Apansson / Vegeta...,1977,5
3,000G1xMMuwxNHmwVsBdtj1,"[[candy_pop, new_wave, new_wave_pop, permanent...","[power_pop, new_wave, rock, candy_pop, new_wav...",Will Anything Happen,32,182347,0,['Blondie'],['4tpUmLEVLCGFr93o8hFFIB'],1978-09-23,...,0.013100,0.000106,0.0821,0.555,191.307,4,[Blondie],BlondieWill Anything Happen,1978,10
4,000KblXP5csWFFFsD6smOy,"[[chamame, folclore_salteno, folklore_argentino]]","[folclore_salteno, folklore_argentino, chamame]",Podrás,35,240013,0,['Tamara Castro'],['11PnNvuse5a0xPupBIeHyW'],2006-07-08,...,0.805000,0.000000,0.1020,0.314,143.757,3,[Tamara_Castro],Tamara_CastroPodrás,2006,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474536,7zzoxJbgjme3366mOp5UnH,"[[dance_pop, pop, pop_rap]]","[pop, pop_rap, dance_pop]",I Took A Pill In Ibiza,58,280800,1,['Mike Posner'],['2KsP6tYLJlTBvSUxnwlVWa'],2016-05-06,...,0.622000,0.000002,0.1250,0.383,74.110,4,[Mike_Posner],Mike_PosnerI Took A Pill In Ibiza,2016,19
474537,7zzqq4eJe4zKpYCHckNi1K,"[[bossa_nova, mpb, nova_mpb]]","[mpb, nova_mpb, bossa_nova]",O Verme E A Estrela,28,191293,0,['Adriana Calcanhotto'],['72f733zGuCPEzCSLs9wOVi'],1994-09-09,...,0.957000,0.015800,0.1100,0.162,109.266,3,[Adriana_Calcanhotto],Adriana_CalcanhottoO Verme E A Estrela,1994,9
474538,7zztLmdxNYrjTQ4XqUK8cp,"[[estonian_pop, estonian_rock]]","[estonian_pop, estonian_rock]",Õhupiraadid,8,163640,0,['Vennaskond'],['4SENxns1ygWdzNSNbBkaXZ'],1993,...,0.000168,0.006220,0.0458,0.668,174.491,4,[Vennaskond],VennaskondÕhupiraadid,1993,2
474539,7zztk8Ecg9qDRLX3XNfSyq,"[[classic_swedish_pop, europop, swedish_pop]]","[classic_swedish_pop, europop, swedish_pop]",Mitt hjärta i din hand,44,215467,0,['Jan Johansen'],['4Nd6R1oljERR3D2893MpJf'],1996-01-01,...,0.343000,0.000000,0.1100,0.409,77.835,4,[Jan_Johansen],Jan_JohansenMitt hjärta i din hand,1996,14


In [27]:
tracks = tracks[tracks['popularity_red']>22]

In [28]:
tracks

,id,genres_upd,consolidates_genre_lists,name,popularity,duration_ms,explicit,artists,id_artists,release_date,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd,artists_song,year,popularity_red
25,001UkMQHw4zXfFNdKpwXAF,"[[contemporary_country, country, country_road,...","[country_road, country_rock, contemporary_coun...",Brand New Man - with Luke Combs,69,191053,0,"['Brooks & Dunn', 'Luke Combs']","['0XKOBt59crntr7HQXXO8Yz', '718COspgdWOnwOFpJH...",2019-04-05,...,0.003720,0.000079,0.2900,0.562,127.061,4,"[Brooks_&_Dunn, Luke_Combs]",Brooks_&_DunnBrand New Man - with Luke Combs,2019,23
54,003FTlCpBTM4eSqYSWPv4H,"[[modern_rock, neo_mellow, pop_punk, pop_rock,...","[pop_punk, pop_rock, post-teen_pop, neo_mellow...","Swing, Swing",69,233267,0,['The All-American Rejects'],['3vAaWhdBR38Q02ohXqaNHT'],2002-10-15,...,0.000065,0.128000,0.0385,0.318,127.947,4,[The_All-American_Rejects],"The_All-American_RejectsSwing, Swing",2002,23
71,003vvx7Niy0yvhvHt4a68B,"[[dance_rock, modern_rock, permanent_wave, rock]]","[rock, modern_rock, dance_rock, permanent_wave]",Mr. Brightside,82,222973,0,['The Killers'],['0C0XlULifJtAgn6ZNCW2eu'],2004,...,0.001210,0.000000,0.0995,0.236,148.033,4,[The_Killers],The_KillersMr. Brightside,2004,27
116,006pMMCuRo2TFjh8sNGSov,"[[brazilian_rock, hard_rock_brasileiro, rock_a...","[rock_nacional_brasileiro, rock_alternativo_br...","Dias De Luta, Dias De Gloria",69,145467,0,['Charlie Brown Jr.'],['1on7ZQ2pvgeQF4vmIA09x5'],2005-01-01,...,0.888000,0.000000,0.1010,0.768,77.963,3,[Charlie_Brown_Jr.],"Charlie_Brown_Jr.Dias De Luta, Dias De Gloria",2005,23
174,009ImBOrIUlWgla8U05RAC,"[[dance_pop, pop, post-teen_pop]]","[pop, dance_pop, post-teen_pop]",Unconditionally,73,228879,0,['Katy Perry'],['6jJ0s89eD6GaHleKKya26X'],2013-01-01,...,0.002730,0.000000,0.2080,0.353,128.902,4,[Katy_Perry],Katy_PerryUnconditionally,2013,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474119,7zXa0myk5r5ex9QfXbjqhY,[[uk_hip_hop]],[uk_hip_hop],Change,73,174316,1,['Alz X 38'],['7nwUxFqqeyUzyYaS7D9lw8'],2019-08-25,...,0.110000,0.000000,0.1030,0.211,95.048,4,[Alz_X_38],Alz_X_38Change,2019,24
474311,7zkLpY72g6lKQbiHDqri1S,"[[adult_standards, contemporary_vocal_jazz, ja...","[adult_standards, jazz_pop, neo_mellow, contem...",Sunrise,73,200627,0,['Norah Jones'],['2Kx7MNY7cI1ENniW7vT30N'],2004-01-01,...,0.941000,0.097000,0.1060,0.679,156.848,4,[Norah_Jones],Norah_JonesSunrise,2004,24
474435,7zsXy7vlHdItvUSH8EwQss,"[[dance_pop, edm, electro_house, moombahton, p...","[danish_pop, pop_rap, indie_poptimism, edm, el...",Cold Water,70,185360,0,"['Major Lazer', 'Justin Bieber', 'MØ']","['738wLrAtLtCtFOLvQBXOXp', '1uNFoZAHBGtllmzznp...",2016-07-22,...,0.073600,0.000000,0.1560,0.501,92.943,4,"[Major_Lazer, Justin_Bieber, MØ]",Major_LazerCold Water,2016,23
474498,7zwn1eykZtZ5LODrf7c0tS,"[[modern_alternative_rock, modern_rock, pop, s...","[pop, modern_alternative_rock, modern_rock, sh...",You Get Me So High,71,153000,0,['The Neighbourhood'],['77SW9BnxLY8rJ0RciFqkHh'],2018-11-02,...,0.186000,0.079100,0.1520,0.387,88.036,4,[The_Neighbourhood],The_NeighbourhoodYou Get Me So High,2018,23


In [29]:
tracks.reset_index(drop = True,inplace = True)

In [30]:
tracks['Song-Artist'] = tracks.apply(lambda row: (row['name'])+" - "+(row['artists_upd'][0]).replace("_"," "),axis = 1)

C:\Users\abhya\AppData\Local\Temp\ipykernel_24316\1964376836.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks['Song-Artist'] = tracks.apply(lambda row: (row['name'])+" - "+(row['artists_upd'][0]).replace("_"," "),axis = 1)


In [31]:
tracks['song'] = tracks['name'].apply(lambda x: "".join(c.lower() for c in x if c.isalpha()))

C:\Users\abhya\AppData\Local\Temp\ipykernel_24316\2446752672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks['song'] = tracks['name'].apply(lambda x: "".join(c.lower() for c in x if c.isalpha()))


In [32]:
tracks['prime-artist'] = tracks['artists_upd'].apply(lambda x: "".join(c.lower() for c in x[0] if c.isalpha()))

C:\Users\abhya\AppData\Local\Temp\ipykernel_24316\3693261358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks['prime-artist'] = tracks['artists_upd'].apply(lambda x: "".join(c.lower() for c in x[0] if c.isalpha()))


In [33]:
tracks.to_pickle('Final_songs_data.pkl')

In [34]:
#simple function to create OHE features
#this gets passed later on
def ohe_prep(df, column, new_name): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [35]:
def create_feature_set(df, float_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    tfidf = TfidfVectorizer(max_features = 500)
    tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = ohe_prep(df, 'year','year')
    popularity_ohe = ohe_prep(df, 'popularity_red','pop')

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns)

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)
     
    #add song id
    final['id']=df['id'].values
    
    return final

In [36]:
complete_feature_set = create_feature_set(tracks, float_cols=float_cols)#.mean(axis = 0)

C:\Users\abhya\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [37]:
complete_feature_set.to_pickle('feature_set_final.pkl')

In [38]:
newdf_tracks = tracks[['id','name','popularity','release_date','artists_upd','year','popularity_red','Song-Artist', 'song', 'prime-artist']]

In [39]:
X = complete_feature_set.drop(['id'], axis = 1).to_numpy()

In [40]:
similarity = cosine_similarity(X)

In [41]:
def recommend1(song, artist = ""):
    if artist != "":
        index = newdf_tracks[(newdf_tracks['song'] == "".join(c for c in song.lower() if c.isalpha())) & (newdf_tracks['prime-artist'] == "".join(c.lower() for c in artist.lower() if c.isalpha()))].index[0]
        distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
        for i in distances[1:30]:
            print(newdf_tracks.iloc[i[0]]['name'] + " by " + newdf_tracks.iloc[i[0]]['artists_upd'][0] )

        
    else:
        index = newdf_tracks[(newdf_tracks['song'] == "".join(c for c in song.lower() if c.isalpha()))]['popularity'].idxmax()
        distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
        for i in distances[1:30]:
            print(newdf_tracks.iloc[i[0]]['name'] + " by " + newdf_tracks.iloc[i[0]]['artists_upd'][0] )


In [52]:
recommend1("Stairway to Heaven - Remaster")

Baba O'Riley by The_Who
Imagine - Remastered 2010 by John_Lennon
American Pie by Don_McLean
Stairway to Heaven - 1990 Remaster by Led_Zeppelin
Going to California - Remaster by Led_Zeppelin
Ain't No Sunshine by Bill_Withers
Dream On by Aerosmith
I Don't Want to Miss a Thing - From "Armageddon" Soundtrack by Aerosmith
Black Dog - Remaster by Led_Zeppelin
Crazy by Aerosmith
Sweet Emotion by Aerosmith
Immigrant Song - Remaster by Led_Zeppelin
Whole Lotta Love - 1990 Remaster by Led_Zeppelin
Walk This Way by Aerosmith
Rock and Roll - Remaster by Led_Zeppelin
Crazy Train by Ozzy_Osbourne
Children of the Grave by Black_Sabbath
Alone by Heart
When the Levee Breaks - Remaster by Led_Zeppelin
Paranoid by Black_Sabbath
I Want to Know What Love Is - 1999 Remaster by Foreigner
Panama - 2015 Remaster by Van_Halen
Pour Some Sugar On Me - Remastered 2017 by Def_Leppard
Eye of the Tiger by Survivor
Walk Of Life by Dire_Straits
Locomotive Breath by Jethro_Tull
La Grange - 2005 Remaster by ZZ_Top
Jealou

In [42]:
newdf_tracks.to_pickle('songs_dict.pkl')

In [55]:
newdf_tracks['name'].values

array(['Brand New Man - with Luke Combs', 'Swing, Swing',
       'Mr. Brightside', ..., 'Some Way', '以後別做朋友', 'Beamer Boi'],
      dtype=object)

In [43]:
pickle.dump(similarity,open("similarity.pkl","wb"))

In [58]:
newdf_tracks

,id,name,popularity,release_date,artists_upd,year,popularity_red,Song-Artist,song,prime-artist
0,001UkMQHw4zXfFNdKpwXAF,Brand New Man - with Luke Combs,69,2019-04-05,"[Brooks_&_Dunn, Luke_Combs]",2019,13,Brand New Man - with Luke Combs - Brooks & Dunn,brandnewmanwithlukecombs,brooksdunn
1,003FTlCpBTM4eSqYSWPv4H,"Swing, Swing",69,2002-10-15,[The_All-American_Rejects],2002,13,"Swing, Swing - The All-American Rejects",swingswing,theallamericanrejects
2,003vvx7Niy0yvhvHt4a68B,Mr. Brightside,82,2004,[The_Killers],2004,16,Mr. Brightside - The Killers,mrbrightside,thekillers
3,005Dlt8Xaz3DkaXiRJgdiS,Historia de Taxi,66,1994-04-14,[Ricardo_Arjona],1994,13,Historia de Taxi - Ricardo Arjona,historiadetaxi,ricardoarjona
4,006pMMCuRo2TFjh8sNGSov,"Dias De Luta, Dias De Gloria",69,2005-01-01,[Charlie_Brown_Jr.],2005,13,"Dias De Luta, Dias De Gloria - Charlie Brown Jr.",diasdelutadiasdegloria,charliebrownjr
...,...,...,...,...,...,...,...,...,...,...
25184,7zwn1eykZtZ5LODrf7c0tS,You Get Me So High,71,2018-11-02,[The_Neighbourhood],2018,14,You Get Me So High - The Neighbourhood,yougetmesohigh,theneighbourhood
25185,7zx1i0jGcFwMBurajgKlO7,Black Magic,62,2020-12-18,[Jonasu],2020,12,Black Magic - Jonasu,blackmagic,jonasu
25186,7zxRMhXxJMQCeDDg0rKAVo,Some Way,72,2017-02-24,"[NAV, The_Weeknd]",2017,14,Some Way - NAV,someway,nav
25187,7zyYmIdjqqiX6kLryb7QBx,以後別做朋友,62,2014-12-19,[Eric_Chou],2014,12,以後別做朋友 - Eric Chou,以後別做朋友,ericchou


In [64]:
newdf_tracks['artists_upd'][0][0]

'Brooks_&_Dunn'

In [68]:
np.asarray([newdf_tracks['artists_upd'][i][0] for i in range(len(newdf_tracks))])

array(['Brooks_&_Dunn', 'The_All-American_Rejects', 'The_Killers', ...,
       'NAV', 'Eric_Chou', 'Khontkar'], dtype='<U51')